# The primary objective of this EDA project is to look for insights and provide guidance for those interested in reaching a top 1000 YouTube channel.
We will be looking into the following:

Which categories are the most popular?

Which categories are the fastest growing?

Which categories are the highest earning?

In [ ]:
# Importing the pandas library and aliasing it as 'pd'
import pandas as pd

# Importing the plotly express module from the plotly library and aliasing it as 'px'
import plotly.express as px

In [ ]:
# Loading the csv into pandas DataFrame
df = pd.read_csv("/kaggle/input/global-youtube-statistics-2023/Global YouTube Statistics.csv", encoding="latin-1")

In [ ]:
# Getting an an overiview of the data
df.info()

In [ ]:
# Finding missing values
df.isnull().sum()

In [ ]:
#looking for duplicates based on 'Title'
duplicate_rows = df[df.duplicated(subset='Title', keep=False)]
duplicate_rows

In [ ]:
# removing duplicates based on 'Title' category
df = df.drop_duplicates(subset='Title')

**Looking at the difference between the columns 'channel_type' and 'category'.
We decide to go with 'category' to use in the analysis due to it having more unique values, allowing for more detail.**

In [ ]:
# Looking at the difference between channel type and channel category
unique_channel_types = df['channel_type'].unique()
unique_categories = df['category'].unique()

print("Unique Channel Types:", unique_channel_types)
print("\nUnique Categories:", unique_categories)

**There are some rows with low or zero yearly earnings.
Channels with this many views should in theory be earning, so we decide to remove these rows because we will be doing analysis on earnings.**

In [ ]:
# drop rows with less than 100 in earnings
df.drop(df[df['highest_yearly_earnings'] < 100].index, inplace=True)
df.drop(df[df['lowest_yearly_earnings'] < 100].index, inplace=True)

In [ ]:
# Displaying the shape of the DataFrame mainly to show row count
df.shape

**We use the average of highest yearly earnings and lowest yearly earnings as a proxy for yearly earnings**

In [ ]:
# We don't have data for every year so we decide to average the highest and lowest yearly earnings estimate
# Create a new column 'yearly_earnings' with the average of 'lowest_yearly_earnings' and 'highest_yearly_earnings'
df['yearly_earnings'] = (df['lowest_yearly_earnings'] + df['highest_yearly_earnings']) / 2

**Looking at minimum and maximun 'created_year'**

In [ ]:
# finding minimum and maximun 'created_year'
min_created_year = df['created_year'].min()
max_created_year = df['created_year'].max()

print(f"Minimum 'created_year': {min_created_year}")
print(f"Maximum 'created_year': {max_created_year}")

**We find there is a channel with the creation date of 1970.**

In [ ]:
# Find the index of the row with the minimum 'created_year'
min_created_year_index = df['created_year'].idxmin()
# display the row
df.loc[min_created_year_index]

**We find that one channel which has a created year of 1970, We decide to drop this row due to the chance that other data in the row could be influenced by the error.**

**For the rest of the data, values of 'created_year' seem reasonable**

In [ ]:
# Drop the rows with the 'created_year' equal to 1970
df = df[df['created_year'] != 1970]
# this drops 1 row

In [ ]:
# Displaying the shape of the DataFrame mainly to show row count
df.shape

**We will be doing analysis using category so we decide to drop rows with missing category values**

In [ ]:
# drop rows where category value is missing
df.dropna(subset=['category'], inplace=True)

**Looking over the data we see some channels with 0 views, this could cause problems when doing statistical analysis, we decide to drop these rows.**

The rows with 0 views seem to be YouTube's own channel categories, which show subscriber count, but views are not shown.

In [ ]:
# drop rows where video views are 0
df = df[df['video views'] != 0]

In [ ]:
# seeing how many rows are remaining for analysis
df.shape

# Relationship Between Views and Subscribers
Here we will be looking at the relationship between views and subscribers, it is shown to be linerarly correlated.

Which seems reasonable, and gives us a clue that the data could be reliable.

Show correlation coefficient

In [ ]:
# Creating a scatter plot using plotly express (px)
fig1 = px.scatter(
    x=df["subscribers"],       # X-axis data from the "subscribers" column of the DataFrame
    y=df["video views"],       # Y-axis data from the "video views" column of the DataFrame
    color=df["category"],       # Color-coding points based on the "category" column of the DataFrame
    size=df["video views"],     # Adjusting point sizes based on the "video views" column of the DataFrame
    hover_name=df["Title"]      # Adding title information for hover-over tooltips based on the "Title" column
)

# Updating the layout of the plot
fig1.update_layout(
    xaxis=dict(title="Subscribers"),  # Adding a title to the X-axis
    yaxis=dict(title="Views"),        # Adding a title to the Y-axis
    title="Relationship between Views and Subscribers",  # Adding a title to the entire plot
    title_x=0.48                      # Adjusting the horizontal position of the title
)

# Displaying the plot
fig1.show()

**Scatterplot showing the relationship between earnings and subscribers**

In [ ]:
# Creating another scatter plot using plotly express (px)
fig2 = px.scatter(
    x=df["subscribers"],           # X-axis data from the "subscribers" column of the DataFrame
    y=df["yearly_earnings"],       # Y-axis data from the "yearly_earnings" column of the DataFrame
    color=df["category"],           # Color-coding points based on the "category" column of the DataFrame
    size=df["yearly_earnings"],     # Adjusting point sizes based on the "yearly_earnings" column of the DataFrame
    hover_name=df["Title"]          # Adding title information for hover-over tooltips based on the "Title" column
)

# Updating the layout of the second plot
fig2.update_layout(
    xaxis=dict(title="Subscribers"),   # Adding a title to the X-axis
    yaxis=dict(title="Earnings"),        # Adding a title to the Y-axis
    title="Relationship between Earnings and Subscribers",
    title_x=0.48
)
# Displaying the plot
fig2.show()

In [ ]:
# Define a pleasant color scheme of 20 colors
colors = [
    'skyblue', 'lightgreen', 'lightcoral', 'gold', 'lightskyblue',
    'lightpink', 'lightgreen', 'lightblue', 'lightcoral', 'lightsalmon',
    'lightseagreen', 'palegoldenrod','paleturquoise', 'lightsteelblue',
    'lightcyan', 'thistle', 'lightcoral', 'lightgreen', 'peachpuff', 'aliceblue'
]

# Distribution of Channels by Category

In [ ]:
# Calculating the count of each category and sorting them in ascending order
category = df['category'].value_counts().sort_values(ascending=True)

# Creating a pie chart using plotly express (px)
fig3 = px.pie(
    values=category.values,          # Values represent the count of each category
    names=category.index             # Names represent the category labels
)

# Updating trace properties for the pie chart
fig3.update_traces(
    textposition='inside',           # Placing text labels inside the pie chart
    textinfo='percent+label'         # Displaying both percentage and label information
)

# Updating the layout of the pie chart
fig3.update_layout(
    title_text="Distribution of Categories",  # Adding a title to the pie chart
    title_x=0.49,                            # Adjusting the horizontal position of the title
    uniformtext_minsize=10,                 # Setting the minimum size for uniform text
    showlegend=False                        # Hiding the legend
)

# Displaying the pie chart
fig3.show()

In [ ]:
# Get the top 12 channel categories
top12_channel_categories = df['category'].value_counts().head(12)

# Plotting the bar chart using plotly.express
fig4 = px.bar(
    x=top12_channel_categories.index,
    y=top12_channel_categories.values,
    color=top12_channel_categories.index,  # Color the bars based on channel categories
    text=top12_channel_categories.values,  # Display the count values on top of the bars
    title='12 Most Popular Channel Categories',
    labels={'x': 'Category', 'y': 'Amount of Channels'}
)

# Customize the layout
fig4.update_layout(
    title_x=0.5,  # Center the title above the plot
    font=dict(size=18),  # Set the font size
    showlegend=False,  # Hide the legend
    xaxis={'categoryorder': 'total ascending'}  # Sort x-axis categories in ascending order
)

# Display the plot
fig4.show()

# Category-Wise Average Subscribers Growth
We see that entertainment and music are popular, but which categories are growing the fastest on average?

In [ ]:
# Calculate Channel Age
df['channel_age'] = 2023 - df['created_year']

# Calculate Average Subscribers growth per Year (asgpy)
df['asgpy'] = df['subscribers']/df['channel_age']

In [ ]:
# Calculate the percentage growth for each category
category_growth = df.groupby('category')['asgpy'].mean().reset_index()

# Calculate the overall mean growth for all categories
overall_mean_growth = df['asgpy'].mean()

# Calculate the percentage growth relative to the overall mean
category_growth['percentage_growth'] = (category_growth['asgpy'] / overall_mean_growth) * 100

# Sort the categories by percentage growth in ascending order and select the top 20
category_growth = category_growth.sort_values(by='percentage_growth', ascending=True).head(20)

# Plotting the bar chart using plotly.express
fig5 = px.bar(category_growth, x='category', y='percentage_growth', color='category',
             color_discrete_sequence=colors, labels={'percentage_growth': 'Percentage Growth Relative to Overall Mean (%)'},
             title='Category-Wise Average Subscribers Growth')

# Center the title above the bar chart
fig5.update_layout(title=dict(x=0.5), showlegend=False)

# Rotate x-axis labels for better readability
fig5.update_layout(xaxis=dict(tickangle=45))

# Set y-axis limits
fig5.update_layout(yaxis=dict(range=[0, 200]))

# Show the plot
fig5.show()

# Highest Yearly Earnings Average by Category

In [ ]:
# Group by 'category' and calculate the mean of yearly earnings for each category
category_earnings = df.groupby('category')['highest_yearly_earnings'].mean().reset_index()

# Sort the categories based on yearly earnings in ascending order
sorted_categories_earnings = category_earnings.sort_values(by='highest_yearly_earnings', ascending=True)

# Choose the top N categories to display
top_categories_earnings = sorted_categories_earnings.head(20)

# Plot the bar chart using plotly.express
fig6 = px.bar(top_categories_earnings, x='category', y='highest_yearly_earnings', 
             color='category', color_discrete_sequence=colors,
             labels={'highest_yearly_earnings': 'Highest Yearly Earnings'},
             title='Highest Yearly Earnings Average by Category')

# Center the title above the bar chart
fig6.update_layout(title=dict(x=0.5), showlegend=False)

# Rotate x-axis labels for better readability
fig6.update_layout(xaxis=dict(tickangle=45))

# Show the plot
fig6.show()

## Approximate Yearly Earnings by Category

In [ ]:
# Create a box plot using plotly.express
fig7 = px.box(df, x='category', y='yearly_earnings', color='category',
             labels={'yearly_earnings': 'Yearly Earnings', 'category': 'Category'},
             title='Approximate Yearly Earnings by Category')

# Center the title above the bar chart
fig7.update_layout(title=dict(x=0.5), showlegend=False)

# Customize the layout for better readability
fig7.update_layout(xaxis=dict(tickangle=45))

# Show the plot
fig7.show()

# Conclusion
The data indicates that 'Shows' and 'Pets & Animals' are the fastest growing categories.

They are also the highest earning.

They are relatively niche (less than 2% of top 1000).

The data indicates that 'Shows' or 'Pets & Animals' would be a strong category to go with if aiming for a top 1000 YouTube channel.

The results could be impacted by insufficient data as we only have 12 channels in the 'shows' category and 2 channels in the 'Pets & Animals' category.